In [5]:
import tensorflow as tf
import keras as kr
from keras import layers
from keras import regularizers
import keras_tuner

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset

In [6]:

df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/MotifsDataAutoencoder/Data/3_46cols_EnvFeatures173samples_plus_3202EC_V2_original.csv",sep=",")

df2 = df.iloc[:,46-len(df.columns):]
#print(df2.head(3))

train_data = np.array(df2)
print(train_data.shape)

inputlen = len(df2.columns)
print(inputlen)


(173, 3202)
3202


# Define model and hyperparameters to be tuned

In [7]:
def autoencoder(units, lr):
    inputlen = 3202
    encoding_dim = 3
    # This is my model
    model = kr.Sequential()
    model.add(kr.Input(shape=(inputlen,)))
    model.add(layers.Dense(units=units*2, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(units=units, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(encoding_dim, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(units=units, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(units=units*2, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(inputlen, activation='linear'))
    
   # This model maps an input to its reconstruction
    autoencoder = kr.Model(model.input, model.output)
    opt = kr.optimizers.Adam(learning_rate=lr)
    autoencoder.compile(optimizer=opt, loss='mse',metrics=["mean_squared_error"])
    return autoencoder

def build_model(hp):
    units = hp.Int("units", min_value=100, max_value=1000, sampling="log")
    lr = hp.Float("lr",min_value=1e-4, max_value=1e-2, sampling="log")
    model = autoencoder(units,lr)
    return model

build_model( keras_tuner.HyperParameters())

# Search the best model

In [8]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="mean_squared_error",
    max_trials=10,
    executions_per_trial=3,
    seed = 333,
    overwrite=True,
    directory="/home/vmh/vmhdocs/Research/Inria/Anl/MotifsDataAutoencoder/Code/autoenc001",
    project_name="tuner",
)


tuner.search_space_summary()
tb = kr.callbacks.TensorBoard(log_dir="./tuner_tb", write_images=True,update_freq="epoch")
tuner.search(train_data, train_data, epochs=20,callbacks=[tb])

Trial 10 Complete [00h 00m 42s]
mean_squared_error: 0.0048256573888162775

Best mean_squared_error So Far: 0.0012419759295880795
Total elapsed time: 00h 03m 03s
INFO:tensorflow:Oracle triggered exit


# Save the best model

In [9]:
# Get best model.
best_model = tuner.get_best_models(num_models=1)
best_model = best_model[0]

# Build the model.
best_model.build(input_shape=(inputlen,))
best_model.summary()

best_hps = tuner.get_best_hyperparameters(1)
best_hps = best_hps[0]
# Build the model with the best hp.
autoencoder = build_model(best_hps)
autoencoder.save('./autoencoder_motifdata.h5')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3202)]            0         
                                                                 
 dense (Dense)               (None, 448)               1434944   
                                                                 
 batch_normalization (BatchN  (None, 448)              1792      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 224)               100576    
                                                                 
 batch_normalization_1 (Batc  (None, 224)              896       
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 3)                 675   